In [1]:
from planning_inference.parsers import parse_model, parse_problem, parse_plan, parse_trajectory, parse_hypothesis, parse_observation_sequence
from planning_inference.generator import generate_trajectory
from planning_inference.functions import generate_all_literals, get_matching_literals

from planning_inference.pddl import Conjunction, Literal, Type, TypedObject, Effect, Truth, NumericConstant, PrimitiveNumericExpression, Increase
from planning_inference.pddl import SensorModel

from planning_inference.observations import Trajectory, Hypothesis, State

from planning_inference import DecodingTask

from sensor_models import load_sensor_model

import os
import copy
from collections import defaultdict
from itertools import combinations
from random import choice, sample, shuffle
import numpy as np
import glob
import time

In [2]:
domain = "openstacks"
task = "prediction"
observability = 50
pnum = 0
timeout = 120

only_correct = True
only_print = True

In [3]:
# Load sensor model
Ms = load_sensor_model(domain)

In [4]:
base_path = "benchmark/%s/%s/%s/" % (domain, task, str(observability))

problems = sorted(glob.glob(base_path + "*"))

problem = problems[pnum]
    
costs = []
times = []

print(problem)

# Planning Model
Mp = parse_model(problem + "/domain")

# Real Hypothesis
with open(problem + "/sol", "r") as f:
    correct_h = int(f.read())

print("Correct hypothesis: %d" % correct_h)

# Observation sequence
obs = parse_observation_sequence(problem + "/obs")

# Hypotheses
h_files = sorted(glob.glob(problem + "/hyp*"))

for i in range(len(h_files)):
    
    if only_correct and correct_h != i:
        continue
    
    h = parse_hypothesis(h_files[i])

    #Build a decoding problem for each hypothesis
    T = DecodingTask(Mp,Ms,obs,h)

    tic = time.time()
    sol = T.decode(clean=True, planner="downward", t=timeout)
    toc = time.time()


    if len(sol.actions) == 0:
        cost = 1000
    else:
        cost = len(sol.actions)
        sol.to_file(problem + "/plan.%s" % str(i).zfill(2))

    duration = toc - tic

    print("Hypothesis %d: %d, %f" % (i, cost, duration))

    costs.append(cost)
    times.append(duration)

if not only_correct and not only_print:
    with open(problem + "/costs", "w") as f:
        f.write(" ".join(map(str, costs)))

    with open(problem + "/times", "w") as f:
        f.write(" ".join(map(str, times)))

benchmark/openstacks/prediction/50/P00
Correct hypothesis: 3
Hypothesis 3: 47, 2.227147
